![Agentic RAG](images/agenticRAG.png)

![Agentic RAG](images/whitePaperVsPractice.png)

The first image shows the flow from the official LangGraph documentation.
If I follow the official documentation, rewrite will not work properly, so I plan to make a slight modification.


In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name = 'income_tax_collection',
    persist_directory = './income_tax_collection'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [2]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str
    
graph_builder = StateGraph(AgentState)

In [4]:
def retrieve(state: AgentState) -> AgentState:
    """
    사용자의 질문에 기반하여 벡터 스토어에서 관련 문서를 검색

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 검색된 문서가 추가된 state를 반환
    """
    query = state['query']  # state에서 사용자의 질문을 추출.
    docs = retriever.invoke(query)  # 질문과 관련된 문서를 검색.
    return {'context': docs}  # 검색된 문서를 포함한 state를 반환.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
from langchain import hub

# RAG 프롬프트를 가져오기
generate_prompt = hub.pull("rlm/rag-prompt")

def generate(state: AgentState) -> AgentState:
    """
    주어진 state를 기반으로 RAG 체인을 사용하여 응답을 생성

    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 생성된 응답을 포함하는 state를 반환
    """
    context = state['context']  # state에서 문맥을 추출
    query = state['query']      # state에서 사용자의 질문을 추출
    
    # RAG 체인을 구성
    rag_chain = generate_prompt | llm
    
    # 질문과 문맥을 사용하여 응답을 생성
    response = rag_chain.invoke({'question': query, 'context': context})
    
    return {'answer': response}  # 생성된 응답을 포함하는 state를 반환

In [ ]:
from langchain import hub
from typing import Literal

# 문서 관련성 판단을 위한 프롬프트를 가져옴.
doc_relevance_prompt = hub.pull("langchain-ai/rag-document-relevance")

def check_doc_relevance(state: AgentState) -> Literal['generate', 'rewrite']:
    """
    주어진 state를 기반으로 문서의 관련성을 판단

    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state.

    Returns:
        Literal['generate', 'rewrite']: 문서가 관련성이 높으면 'generate', 그렇지 않으면 'rewrite'를 반환
    """
    query = state['query']  # state에서 사용자의 질문을 추출
    context = state['context']  # state에서 문맥을 추출

    # 문서 관련성 판단 체인을 구성
    doc_relevance_chain = doc_relevance_prompt | llm

    # 질문과 문맥을 사용하여 문서의 관련성을 판단
    response = doc_relevance_chain.invoke({'question': query, 'documents': context})

    # 관련성이 높으면 'generate'를 반환하고, 그렇지 않으면 'rewrite'를 반환
    if response['Score'] == 1:
        return 'generate'
    
    return 'rewrite'